In [0]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/dl-security/') #Change the path to the directory that contains all code and data

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#!pip install https://github.com/bethgelab/foolbox/archive/master.zip

In [4]:
from bagnets.utils import plot_heatmap, generate_heatmap_pytorch
from bagnets.utils import pad_image, convert2channel_last, imagenet_preprocess, extract_patches, bagnet_predict, compare_heatmap
from bagnets.utils import bagnet33_debug, plot_saliency, compute_saliency_map
from bagnets.utils import get_topk_acc, validate
from foolbox.utils import samples
import bagnets.pytorch
from bagnets.pytorch import Bottleneck
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
import time
import os
img_path = "./ILSVRC2012_img_val"
root = "./"
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
if use_cuda:
    print(torch.cuda.get_device_name(0))

Tesla T4


## Define clipping functions

In [5]:
bagnet33 = bagnets.pytorch.bagnet33(pretrained=True, avg_pool=False).to(device)
bagnet33.eval()
print()

In [0]:
def clip_pm1(values, **kwargs):
    """Clip values to [-1, 1]
    Input:
    - values(torch tensor): values to be clipped
    Output: (torch tensor) clipped values
    """
    return torch.clamp_(values, -1., 1.)

def clip_bias(values, b):
    """Clip values to [-1, 1]
    Input:
    - values(torch tensor): values to be clipped
    - b (float): intersection
    Output: (torch tensor) clipped values
    """
    return torch.clamp_(values + b, -1, 1)

def clip_linear(values, a, b):
    """Clip values to [-1, 1]
    Input:
    - values(torch tensor): values to be clipped
    - a (float): coefficient
    - b (float): intersection
    Output: (torch tensor) clipped values
    """
    return torch.clamp_(values * a + b, -1, 1)

def sigmoid_linear(values, a, b):
    """Clip values to [-1, 1]
    Input:
    - values(torch tensor): values to be clipped
    - a (float): coefficient
    - b (float): intersection
    Output: (torch tensor) clipped values
    """
    x_lin = values * a + b
    return torch.sigmoid(x_lin)

def tanh_linear(values, a, b):
    """Clip values to [-1, 1]
    Input:
    - values(torch tensor): values to be clipped
    - a (float): coefficient
    - b (float): intersection
    Output: (numpy array) clipped values
    """
    return torch.tanh(values * a + b)

In [0]:
def clip_logits(bagnet, clip, images, **kwargs):
    """Clip logits returned by patches
    Input:
    - bagnet (pytorch model): Bagnet without average pooling
    - clip (python function): clip function
    - images (pytorch tensor): 
    """
    with torch.no_grad():
        patch_logits = bagnet(images)
    return clip(patch_logits, **kwargs)

In [0]:
bs = 20
original, labels = samples(dataset='imagenet', index=1, batchsize=bs, shape=(224, 224), data_format='channels_first')
images = imagenet_preprocess(original)
images, targets = torch.from_numpy(images).to(device), torch.from_numpy(labels).to(device)

In [0]:
pm1_clip = clip_logits(bagnet33, clip_pm1, images)
bias_clip = clip_logits(bagnet33, clip_bias, images, b=10)
linear_clip = clip_logits(bagnet33, clip_linear, images, a=0.5, b=10)
sigmoid_clip = clip_logits(bagnet33, sigmoid_linear, images, a=0.5, b=10)
tanh_clip = clip_logits(bagnet33, tanh_linear, images, a=0.5, b=10)